In [1]:
!pip install transformers

In [ ]:
from google.colab import drive
import IPython.display as ipd
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from pathlib import Path
import sklearn
from transformers import AutoFeatureExtractor #generador de características


drive.mount("/content/drive")
%cd "/content/drive/MyDrive/UChile/ML"

Cargando señales de audio

In [ ]:
MyPath = pathlib.Path("/content/drive/MyDrive/UChile/ML/02 - Introducción a la programación en python/archive")

Necesitamos utilizar un sample rate menor para ejemplificar utilizando transformers

In [ ]:
sr = 16000

In [ ]:
kick_signals = [
    librosa.load(p, sr = sr)[0] for p in MyPath.glob('kick/Bass Sample *.wav')
]

snare_signals = [
    librosa.load(p, sr = sr)[0] for p in MyPath.glob('snare/Snare Sample *.wav')
]

Cargando un modelo que extrae características desde las señales de audio

In [ ]:
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

In [ ]:
kick_inputs = feature_extractor(kick_signals, sampling_rate=sr, return_tensors="pt")

In [ ]:
snare_inputs = feature_extractor(snare_signals, sampling_rate=sr, return_tensors="pt")

In [ ]:
len(kick_inputs)

In [ ]:
kick_inputs['input_values'].shape

Obteniendo los arreglos que contienen la información que el modelo consiguió extraer del audio

In [ ]:
kick_embeddings = kick_inputs['input_values']

In [ ]:
snare_embeddings = snare_inputs['input_values']

Concatenamos los tensores-vectores en ```torch```

In [ ]:
import torch

In [ ]:
X = torch.cat((kick_embeddings, snare_embeddings), 0)

In [ ]:
X.shape

Aplicamos $t-\mathtt{SNE}$

In [ ]:
from sklearn import manifold

In [ ]:
tsne = manifold.TSNE(n_components = 2, random_state = 42)

In [ ]:
transformed_data = tsne.fit_transform(X)

In [ ]:
transformed_data.shape

In [ ]:
import pandas as pd

In [ ]:
df_tsne = pd.DataFrame(transformed_data)
df_tsne['targets'] = 40*[1] + 40*[0]

In [ ]:
df_tsne.columns = ['x','y','targets']

In [ ]:
df_tsne.head(5)

In [ ]:
import seaborn as sns

In [ ]:
grid = sns.FacetGrid(df_tsne,hue = "targets")
grid.map(plt.scatter,"x","y").add_legend()